https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-11-18 05:03:51--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.ta

In [ ]:
import pandas as pd
import numpy as np
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
from collections import Counter # 단어 등장 횟수 카운트

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
dataset = pd.read_csv("/content/oasis_market.csv", encoding='utf-8')
dataset = dataset.drop('Unnamed: 0', axis=1)
dataset

,title,datetime,content
0,오아시스마켓 3만원으로 일주일 장보기 3인가족,2023. 11. 3. 14:48,오아시스마켓 3만원으로 일주일 장보기 3인가족 요즘 식비 절약하면서 장을 봐서 반찬...
1,오아시스마켓 아이스홍시 추천,2023. 10. 25. 18:05,"오아시스마켓 아이스홍시 추천 오아시스마켓, 대전은 새벽 배송 안됨 오아시스마켓 대전..."
2,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인,2023. 10. 16. 21:59,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인 ...
3,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템,2023. 9. 15. 15:00,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템 오아시스마켓 돼지고기 추천 농수산물...
4,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이",2023. 9. 30. 9:10,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이 당장 먹을 게 없다..."
...,...,...,...
1013,초기이유식 토핑이유식 큐브 :: 양배추,2023. 10. 18. 14:36,초기이유식 토핑이유식 큐브 :: 양배추 양배추는 질산염이 많은 채소라\n초기이유식에...
1014,구운 방사유정란 동물복지유정란 건강한 간식,2023. 6. 7. 0:00,구운 방사유정란 동물복지유정란 건강한 간식 배고플 때 쓸데없는 거 군것질 안하고 훈...
1015,코스트코 쇼핑 목록 우리 집 쟁여템 소개,2023. 11. 13. 16:14,"코스트코 쇼핑 목록 우리 집 쟁여템 소개 코스트코 11월\n할인상품, 쟁여템 202..."
1016,6월 첫째주 일주일식단 :: 냉털의 시기다,2023. 6. 11. 20:15,6월 첫째주 일주일식단 :: 냉털의 시기다 오아시스마켓이며 마켓컬리에서 쿠폰을 많이...


In [ ]:
# txt 파일에서 stopword를 읽어오기
with open('/content/stopwords.txt', 'r', encoding='UTF-8') as file:
    stopword_data = file.read()


# 쉼표로 구분된 데이터를 리스트로 변환
stopword_list = stopword_data.split(',')
stopword_list = pd.DataFrame(stopword_list)
stopword_list.head()

,0
0,휴
1,아이구
2,아이쿠
3,아이고
4,어


In [ ]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['content']))
dataset['content_removed'] = review_removed
dataset

,title,datetime,content,content_removed
0,오아시스마켓 3만원으로 일주일 장보기 3인가족,2023. 11. 3. 14:48,오아시스마켓 3만원으로 일주일 장보기 3인가족 요즘 식비 절약하면서 장을 봐서 반찬...,오아시스마켓 만원으로 일주일 장보기 인가족 요즘 식비 절약하면서 장을 봐서 반찬을 ...
1,오아시스마켓 아이스홍시 추천,2023. 10. 25. 18:05,"오아시스마켓 아이스홍시 추천 오아시스마켓, 대전은 새벽 배송 안됨 오아시스마켓 대전...",오아시스마켓 아이스홍시 추천 오아시스마켓 대전은 새벽 배송 안됨 오아시스마켓 대전 ...
2,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인,2023. 10. 16. 21:59,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인 ...,월 주 차 오아시스마켓장보기 일주일식단표 도시락메뉴 오아시스마켓추천인 직장인인 나...
3,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템,2023. 9. 15. 15:00,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템 오아시스마켓 돼지고기 추천 농수산물...,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템 오아시스마켓 돼지고기 추천 농수산물...
4,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이",2023. 9. 30. 9:10,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이 당장 먹을 게 없다...",새벽배송 오아시스마켓 후다닥 차린 아침밥상 황태국과 조기구이 당장 먹을 게 없다그럴...
...,...,...,...,...
1013,초기이유식 토핑이유식 큐브 :: 양배추,2023. 10. 18. 14:36,초기이유식 토핑이유식 큐브 :: 양배추 양배추는 질산염이 많은 채소라\n초기이유식에...,초기이유식 토핑이유식 큐브 양배추 양배추는 질산염이 많은 채소라초기이유식에 사용해...
1014,구운 방사유정란 동물복지유정란 건강한 간식,2023. 6. 7. 0:00,구운 방사유정란 동물복지유정란 건강한 간식 배고플 때 쓸데없는 거 군것질 안하고 훈...,구운 방사유정란 동물복지유정란 건강한 간식 배고플 때 쓸데없는 거 군것질 안하고 훈...
1015,코스트코 쇼핑 목록 우리 집 쟁여템 소개,2023. 11. 13. 16:14,"코스트코 쇼핑 목록 우리 집 쟁여템 소개 코스트코 11월\n할인상품, 쟁여템 202...",코스트코 쇼핑 목록 우리 집 쟁여템 소개 코스트코 월할인상품 쟁여템 년 월 일 토요...
1016,6월 첫째주 일주일식단 :: 냉털의 시기다,2023. 6. 11. 20:15,6월 첫째주 일주일식단 :: 냉털의 시기다 오아시스마켓이며 마켓컬리에서 쿠폰을 많이...,월 첫째주 일주일식단 냉털의 시기다 오아시스마켓이며 마켓컬리에서 쿠폰을 많이 주길...


In [ ]:
review_tokenized = dataset['content_removed'].apply(lambda review: okt.nouns(review))

In [ ]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list[0]):
                review_removed_stopword.append(token)

    return review_removed_stopword

In [ ]:
dataset['content_removed_stopword'] = review_tokenized.apply(remove_stopword).replace(",", "")
content_removed_stopword = dataset['content_removed_stopword']

In [ ]:
dataset['content_removed_stopword'] = content_removed_stopword
dataset

,title,datetime,content,content_removed,content_removed_stopword
0,오아시스마켓 3만원으로 일주일 장보기 3인가족,2023. 11. 3. 14:48,오아시스마켓 3만원으로 일주일 장보기 3인가족 요즘 식비 절약하면서 장을 봐서 반찬...,오아시스마켓 만원으로 일주일 장보기 인가족 요즘 식비 절약하면서 장을 봐서 반찬을 ...,"[장보기, 가족, 식비, 절약, 반찬, 반찬, 양하, 단백질, 야채, 단식, 유지,..."
1,오아시스마켓 아이스홍시 추천,2023. 10. 25. 18:05,"오아시스마켓 아이스홍시 추천 오아시스마켓, 대전은 새벽 배송 안됨 오아시스마켓 대전...",오아시스마켓 아이스홍시 추천 오아시스마켓 대전은 새벽 배송 안됨 오아시스마켓 대전 ...,"[아이스, 홍시, 추천, 대전, 배송, 대전, 배송, 아침, 일곱시, 다기, 곧장,..."
2,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인,2023. 10. 16. 21:59,[10월 3주 차] 오아시스마켓-장보기 일주일식단표 도시락메뉴 오아시스마켓추천인 ...,월 주 차 오아시스마켓장보기 일주일식단표 도시락메뉴 오아시스마켓추천인 직장인인 나...,"[장보기, 식단, 도시락, 추천, 직장인, 평일, 식단, 금요일, 대략, 정리, 물..."
3,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템,2023. 9. 15. 15:00,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템 오아시스마켓 돼지고기 추천 농수산물...,오아시스마켓 돼지고기 추천 농수산물할인쿠폰 득템 오아시스마켓 돼지고기 추천 농수산물...,"[돼지고기, 추천, 수산물, 할인, 쿠폰, 득템, 돼지고기, 추천, 수산물, 할인,..."
4,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이",2023. 9. 30. 9:10,"새벽배송 오아시스마켓., 후다닥 차린 아침밥상 황태국과 조기구이 당장 먹을 게 없다...",새벽배송 오아시스마켓 후다닥 차린 아침밥상 황태국과 조기구이 당장 먹을 게 없다그럴...,"[배송, 후다닥, 아침밥, 태국, 조기, 구이, 당장, 배송, 배송, 후다닥, 아침..."
...,...,...,...,...,...
1013,초기이유식 토핑이유식 큐브 :: 양배추,2023. 10. 18. 14:36,초기이유식 토핑이유식 큐브 :: 양배추 양배추는 질산염이 많은 채소라\n초기이유식에...,초기이유식 토핑이유식 큐브 양배추 양배추는 질산염이 많은 채소라초기이유식에 사용해...,"[초기, 이유식, 토핑, 이유식, 큐브, 양배추, 양배추, 질산염, 채소, 초기, ..."
1014,구운 방사유정란 동물복지유정란 건강한 간식,2023. 6. 7. 0:00,구운 방사유정란 동물복지유정란 건강한 간식 배고플 때 쓸데없는 거 군것질 안하고 훈...,구운 방사유정란 동물복지유정란 건강한 간식 배고플 때 쓸데없는 거 군것질 안하고 훈...,"[방사, 유정, 동물복지, 유정, 군것질, 훈제, 계란, 유정, 홈플러스, 훈제란,..."
1015,코스트코 쇼핑 목록 우리 집 쟁여템 소개,2023. 11. 13. 16:14,"코스트코 쇼핑 목록 우리 집 쟁여템 소개 코스트코 11월\n할인상품, 쟁여템 202...",코스트코 쇼핑 목록 우리 집 쟁여템 소개 코스트코 월할인상품 쟁여템 년 월 일 토요...,"[코스트코, 쇼핑, 목록, 쟁여템, 코스트코, 할인, 상품, 쟁여템, 토요일, 코코..."
1016,6월 첫째주 일주일식단 :: 냉털의 시기다,2023. 6. 11. 20:15,6월 첫째주 일주일식단 :: 냉털의 시기다 오아시스마켓이며 마켓컬리에서 쿠폰을 많이...,월 첫째주 일주일식단 냉털의 시기다 오아시스마켓이며 마켓컬리에서 쿠폰을 많이 주길...,"[식단, 냉털, 시기, 쿠폰, 냉장고, 사지, 냉털, 냉장고, 반찬, 냉털, 반찬,..."


In [ ]:
# 각 내부 리스트의 모든 요소를 하나의 문자열로 합친 후 리스트 생성
output_list = [' '.join(item) for item in content_removed_stopword]

# 데이터프레임에 새로운 칼럼 추가
dataset['new_column'] = output_list
new_column = dataset['new_column']

In [ ]:
documents = list()
documents = new_column.values.tolist()
print(documents[:5])

['장보기 가족 식비 절약 반찬 반찬 양하 단백질 야채 단식 유지 반찬 칼로리 음식 쭉쭉 뱃살 몸무게 사이트 배송 배송 먹거리 쿠폰 제주 청정 우유 한우 곰탕 항생제 국산 순두부 송이버섯 항생제 통닭 두부 멸치 액젓 완도 다시마 농약 깻잎 합계 쿠폰 결제 장본 음식 레시피 곰탕 순두부찌개 소스 버섯 볶음 간장 두부 조림', '아이스 홍시 추천 대전 배송 대전 배송 아침 일곱시 다기 곧장 아침 식탁 대전 당일 배송 배송 계란 난각 번호 유정 팝콘 치킨 현미퐁 수프 아이스 홍시 약과 유정 선물 항생제 우유 결제 인기 상품 우유 다섯시 결제 출고 당일 당일 출고 토스 카드 결제 결제 토스 결제 토스 포인트 다섯시 열시 택배 택배 걱정 계란 액체 스티로폼 물건 계란 비닐 완충 번란 마트 자유 프라이 번란 초코 현미 플레인 요구르트 포장 초코 현미 현미퐁 초코 부피 달달 초코 신한 현미 어른 국산 밀가루 약과 대여섯 입일 작고 약과 번쩍 미니 약과 주방 세제 개구리 세제 냉동식품 비닐 드라이아이스 벽돌 드라이아이스 음식물 걱정 우유 선물 우유 모나 슬림 플라스틱 냉장고 포켓 도어 계획 팝콘 치킨 닭고기 기름 음식 요긴 감자 수프 아침 모닝 중탕 전자레인지 카레 밀가루 반죽 아이스 홍시 딸아이 급식 소리 학교 급식 아이스 홍시 요구르트 낱개 포장 냉동 상태 냉장 아이스 홍시 달달 아이스크림 첨가 홍시 과일 안쪽 씨앗 겉껍질 상태 숟가락 애나 어른 두루 추천 쿠폰 추천 쿠폰 배송 대전 배송 홍시 추천 물품 초코 현미 입약 추천', '장보기 식단 도시락 추천 직장인 평일 식단 금요일 대략 정리 물품 토요일 배송 래야 주말 손질 평일 후다닥 반찬 주말 반찬 도시락 반찬 신선도 저녁 도시락 대략 식단 구성 식단 장도 손질 면조 금은 부담 칭찬 아두 장보기 쿠폰 쿠폰 출장 주말 예약 배송 계란 부도 쿠폰 아두 예약 배송 당장 물품 배송 할인 쿠폰 할인 쿠폰 쿠폰 당일 쿠폰 당일 날짜 예약 배송 당장 물품 예약 배송 날짜 배송 필수품 계란 두부 도시락 식단 도시락 냉장고 장보기 감

In [ ]:
len(documents)

1018

In [ ]:
vectorizer = CountVectorizer(max_features=10)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(1018, 2000)


In [ ]:
vocab = vectorizer.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
len(vocab)

2000

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=40, num_epochs=70)
ctm.fit(training_dataset)

Epoch: [50/50]	 Seen Samples: [50900/50900]	Train Loss: 715.2677814955796	Time: 0:00:00.541535: : 50it [00:32,  1.53it/s]


features=3000 Epoch =30	Train Loss: 840.2390333374263\
features=2000 Epoch =30	Train Loss: 748.5771503560903\
features=2000 Epoch =50	Train Loss: 715.2677814955796

In [ ]:
ctm.get_topics()

defaultdict(list,
            {0: ['위생', '대체로', '쥬스', '화면', '인정', '오기', '초록색', '장본', '이동', '순이'],
             1: ['편입', '실수', '보냉', '대략', '초록색', '주차장', '핑계', '오기', '이사', '고소'],
             2: ['뚜껑', '마늘', '개봉', '오븐', '숟가락', '사랑', '반죽', '첫째', '래미', '분량'],
             3: ['추천', '장보기', '농약', '제품', '식비', '외식', '야채', '두부', '포장', '가계부'],
             4: ['치즈', '마늘', '저녁', '소스', '국산', '우유', '소고기', '샐러드', '아침', '양파'],
             5: ['할인', '신청', '쿠폰', '혜택', '축산물', '키트', '행사', '추석', '시장', '전통'],
             6: ['기업',
              '서비스',
              '친환경',
              '사업',
              '식품',
              '상품',
              '뱅크',
              '매출',
              '농산물',
              '진행'],
             7: ['추천',
              '쿠폰',
              '할인',
              '배송',
              '가계부',
              '가격',
              '포인트',
              '결제',
              '장보기',
              '제품'],
             8: ['추천', '계란', '두부', '야채', '채소', '제품', '가격', '어묵', '당근', '양파'],
            

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Streaming output truncated to the last 5000 lines.
  if type(self.X_bow[i]) == scipy.sparse.csr.csr_matrix:
/usr/local/lib/python3.10/dist-packages/contextualized_topic_models/datasets/dataset.py:32: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if type(self.X_bow[i]) == scipy.sparse.csr.csr_matrix:
/usr/local/lib/python3.10/dist-packages/contextualized_topic_models/datasets/dataset.py:32: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if type(self.X_bow[i]) == scipy.sparse.csr.csr_matrix:
/usr/local/lib/python3.10/dist-packages/contextualized_topic_models/datasets/dataset.py:32: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if type(self.X_bow[i]) == scipy.sparse.csr.csr_matrix:
/usr/local/lib/python3.10/dist-packages/contextualized_topic_mod

BrokenProcessPool: ignored

In [ ]:
def print_topic_prop(topics, RATING):
    topic_values = []
    for topic in topics:
        topic_value = topic[1]
        topic_values.append(topic_value)
    topic_prop = pd.DataFrame({"topic_num" : list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})
    topic_prop.to_excel('./result/topic_prop_' + RATING +  '.xlsx')
    display(topic_prop)

In [ ]:
RATING = 'pos'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)